In [147]:
import pandas as pd

In [148]:
# Load the dataset
user = "aldi"  # Replace with the actual username
file_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/combined_davis.xlsx"  
df = pd.read_excel(file_path)

In [149]:
# Loop through the DataFrame and mark rows for deletion
to_delete = []
for i in range(len(df) - 1):
    # Check for consecutive rows with "Sam Groth" as "Player 1" and "John Peers" as "Player 2"
    if (df.iloc[i]['Player 1'] == "Sam Groth" and 
        df.iloc[i]['Player 2'] == "John Peers" and
        df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1'] and
        df.iloc[i]['Player 2'] == df.iloc[i + 1]['Player 2']):
        to_delete.append(i + 1)
    
    # Check for consecutive rows with "Jordan Thompson" as "Player 1"
    elif (df.iloc[i]['Player 1'] == "Jordan Thompson" and 
          df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1']):
        to_delete.append(i + 1)

    # Check for consecutive rows with "Nick Kyrgios" as "Player 1"
    elif (df.iloc[i]['Player 1'] == "Nick Kyrgios" and 
          df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1']):
        to_delete.append(i + 1)
        
    # Check for consecutive rows with "Sam Groth" as "Player 1"
    elif (df.iloc[i]['Player 1'] == "Sam Groth" and 
          df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1']):
        to_delete.append(i + 1)
    # Check for consecutive rows with "Sam Groth" as "Player 1" and "John Peers" as "Player 2"
    elif (df.iloc[i]['Player 1'] == "John Peers" and 
        df.iloc[i]['Player 2'] == "Jordan Thompson" and
        df.iloc[i]['Player 1'] == df.iloc[i + 1]['Player 1'] and
        df.iloc[i]['Player 2'] == df.iloc[i + 1]['Player 2']):
        to_delete.append(i + 1)


# Remove the marked rows
df_cleaned = df.drop(df.index[to_delete])


In [150]:
# Extract the relevant columns and rename them
df_extracted = df_cleaned[['Stage', 'Player 1', 'Player 2', 'Year', 'Date', 'Venue', 'Court Pace Rating', 'Ball', 'match']].copy()
df_extracted.rename(columns={'Player 1': 'P1T1', 'Player 2': 'P2T1'}, inplace=True)

# Helper function to pair rows with the additional condition that "Date" should be equal
def pair_rows(df):
    paired_rows = []
    for i in range(0, len(df), 2):
        if i+1 < len(df):
            row1 = df.iloc[i]
            row2 = df.iloc[i+1]
            if row1['match'] == row2['match'] and row1['Year'] == row2['Year'] and row1['Date'] == row2['Date']:
                paired_row = [row1['match'], row1['Year'], row1['P1T1'], row1['P2T1'], row2['P1T1'], row2['P2T1'], row1['Date'], row1['Venue'], row1['Court Pace Rating'], row1['Ball']]
                paired_rows.append(paired_row)
    paired_df = pd.DataFrame(paired_rows, columns=['Match', 'Year', 'P1T1', 'P2T1', 'P1T2', 'P2T2', 'Date', 'Venue', 'Court Pace Rating', 'Ball'])
    return paired_df

# Pair the rows in the extracted dataframe
df_paired = pair_rows(df_extracted)

# Print the shapes of the dataframes
print("df shape:", df.shape)
print("df_cleaned shape:", df_cleaned.shape)
print("df_paired shape:", df_paired.shape)


df shape: (866, 28)
df_cleaned shape: (846, 28)
df_paired shape: (423, 10)


# ! too few pairs - pair function not working correctly

In [164]:
# Group by Date, Year, and Venue, then count observations
grouped_counts = df_cleaned.groupby(['Date', 'Year', 'Venue']).size()

# Define a function to check if counts meet the specified criteria
def check_count(year, count):
    if year <= 2018:
        return count == 8 or count == 10
    else:
        return 4 <= count <= 16

# Apply the function to each row in the grouped data and create a new column 'Count'
correct_counts_df = grouped_counts.reset_index()
correct_counts_df['Count'] = correct_counts_df.apply(lambda row: row[0], axis=1)

# Filter rows where counts do not meet the criteria
incorrect_counts_df = correct_counts_df[~correct_counts_df.apply(lambda row: check_count(row['Year'], row['Count']), axis=1)]

# Check if the DataFrame is empty, indicating all matches are in the correct range
if incorrect_counts_df.empty:
    print("All matches are in the correct range.")
else:
    # Display Date, Year, Venue, and Count when correct_counts is False
    print(incorrect_counts_df[['Date', 'Year', 'Venue', 'Count']])


All matches are in the correct range.


In [152]:
# Convert 'Year' to integer
df_cleaned['Year'] = df_cleaned['Year'].astype(int)

# Filter observations based on year, Date, and Venue containing "Melbourne"
filtered_obs = df_cleaned[
    (df_cleaned['Year'] == 2017) &
    (df_cleaned['Date'] == " 03 Feb - 05 Feb 2017") &
    (df_cleaned['Venue'].str.contains("Melbourne"))
]

# Display the filtered observations
print(filtered_obs[['Player 1', 'Player 2']])

            Player 1     Player 2
440  Jordan Thompson          NaN
442      Jiri Vesely          NaN
443     Nick Kyrgios          NaN
445       Jan Satral          NaN
446        Sam Groth   John Peers
450       Jan Satral  Jiri Vesely
451        Sam Groth          NaN
453      Jiri Vesely          NaN
454  Jordan Thompson          NaN
456       Jan Satral          NaN


In [153]:
# keep only rows for single matches
davis_single_df = df_paired[df_paired['P2T1'].isna() & df_paired['P2T2'].isna()]
print("davis_single_df", davis_single_df.shape)

davis_single_df (331, 10)


In [154]:
# Replace specific player names to match with data from atp matches
davis_single_df.replace({
    'P1T1': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos'
    },
    'P2T1': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos'
    },
    'P1T2': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos'
    },
    'P2T2': {
        'Jan-Lennard Struff': 'Jan Lennard Struff',
        'Juan Martin Del Potro': 'Juan Martin del Potro',
        'Albert Ramos-Vinolas': 'Albert Ramos'
    }
}, inplace=True)




C:\Users\aldi\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [155]:
davis_single_df.head(20)

,Match,Year,P1T1,P2T1,P1T2,P2T2,Date,Venue,Court Pace Rating,Ball
0,MATCH 1,2014,Radek Stepanek,NaN,Robin Haase,NaN,31 Jan - 02 Feb 2014,"Cez Arena, Ostrava, Czech Republic",Medium,Head ATP
1,MATCH 2,2014,Tomas Berdych,NaN,Igor Sijsling,NaN,31 Jan - 02 Feb 2014,"Cez Arena, Ostrava, Czech Republic",Medium,Head ATP
3,MATCH 4,2014,Tomas Berdych,NaN,Thiemo De Bakker,NaN,31 Jan - 02 Feb 2014,"Cez Arena, Ostrava, Czech Republic",Medium,Head ATP
4,MATCH 5,2014,Lukas Rosol,NaN,Igor Sijsling,NaN,31 Jan - 02 Feb 2014,"Cez Arena, Ostrava, Czech Republic",Medium,Head ATP
5,MATCH 1,2014,Kei Nishikori,NaN,Peter Polansky,NaN,31 Jan - 02 Feb 2014,"Ariake Coliseum, Tokyo, Japan",Medium,Srixon
6,MATCH 2,2014,Go Soeda,NaN,Frank Dancevic,NaN,31 Jan - 02 Feb 2014,"Ariake Coliseum, Tokyo, Japan",Medium,Srixon
8,MATCH 4,2014,Kei Nishikori,NaN,Frank Dancevic,NaN,31 Jan - 02 Feb 2014,"Ariake Coliseum, Tokyo, Japan",Medium,Srixon
9,MATCH 5,2014,Go Soeda,NaN,Peter Polansky,NaN,31 Jan - 02 Feb 2014,"Ariake Coliseum, Tokyo, Japan",Medium,Srixon
10,MATCH 1,2014,Philipp Kohlschreiber,NaN,Roberto Bautista Agut,NaN,31 Jan - 02 Feb 2014,"Fraport Arena, Frankfurt, Germany",Medium Fast,Dunlop Fort Tournament
11,MATCH 2,2014,Florian Mayer,NaN,Feliciano Lopez,NaN,31 Jan - 02 Feb 2014,"Fraport Arena, Frankfurt, Germany",Medium Fast,Dunlop Fort Tournament


In [156]:
# Initialize DataFrame to store all ATP matches where Davis Cup players took part
combined_matches_df = pd.DataFrame()

for year in range(2014, 2024):
    # Filter Davis Cup data for the current year
    davis_single_year_df = df_paired[df_paired['Year'] == year]

    # Create a set of tuples representing pairs of players for the current year
    davis_pairs = set(zip(davis_single_year_df['P1T1'], davis_single_year_df['P1T2']))

    # Load ATP matches data for the current year
    atp_matches_path = f"{folder_path}atp_matches_{year}.csv"
    atp_matches_df = pd.read_csv(atp_matches_path)

    # Filter ATP matches data to match Davis Cup pairs
    matched_atp_matches_df = atp_matches_df[
        atp_matches_df.apply(lambda row: (row['winner_name'], row['loser_name']) in davis_pairs or
                                           (row['loser_name'], row['winner_name']) in davis_pairs, axis=1)
    ]

    # Append the filtered data to the combined DataFrame
    combined_matches_df = combined_matches_df.append(matched_atp_matches_df)

# Extract the year information before the first hyphen in 'tourney_id'
combined_matches_df['year'] = combined_matches_df['tourney_id'].str.split('-').str[0]
print("combined_matches_df", combined_matches_df.shape)
# Save the combined data to a single Excel file
final_output_path = f"C:/Users/{user}/Documents/GitHub/davis-cup/data/matches/filtered_atp_matches.xlsx"
combined_matches_df.to_excel(final_output_path, index=False)


combined_matches_df (414, 50)


In [157]:
# Count the number of observations where tourney_level is "D" - Davis Cup
count_tourney_level_D = combined_matches_df[combined_matches_df['tourney_level'] == 'D'].shape[0]

# Count the number of observations where tourney_level is not "D"
count_not_tourney_level_D = combined_matches_df[combined_matches_df['tourney_level'] != 'D'].shape[0]

print("Number of observations with tourney_level 'D':", count_tourney_level_D)
print("Number of observations without tourney_level 'D':", count_not_tourney_level_D)


Number of observations with tourney_level 'D': 310
Number of observations without tourney_level 'D': 104


In [158]:
# Creating a new DataFrame with observations where tourney_level is "D"
davis_cup_matches_df = combined_matches_df[combined_matches_df['tourney_level'] == 'D']


In [159]:

# Creating a list with year, match_num, winner_name, loser_name, where names are alphabetically ordered
matches_list = davis_cup_matches_df.apply(
    lambda row: [row['year'], *sorted([row['winner_name'], row['loser_name']])], 
    axis=1
).tolist()

# Optionally, print the first few elements in the list to verify
print(matches_list[:5])


[['2014', 'Andreas Seppi', 'Carlos Berlocq'], ['2014', 'Fabio Fognini', 'Juan Monaco'], ['2014', 'Carlos Berlocq', 'Fabio Fognini'], ['2014', 'Radek Stepanek', 'Robin Haase'], ['2014', 'Igor Sijsling', 'Tomas Berdych']]


In [160]:
# Clean the 'Match' column to keep only the number after "MATCH"
davis_single_df['Match'] = davis_single_df['Match'].str.replace('MATCH ', '').astype(int)

# Creating a list with Year, cleaned Match, P1T1, P1T2, where player names are alphabetically ordered
davis_single_list = davis_single_df.apply(
    lambda row: [row['Year'], *sorted([row['P1T1'], row['P1T2']])], 
    axis=1
).tolist()

# Optionally, print the first few elements in the list to verify
print(davis_single_list[:5])


[[2014, 'Radek Stepanek', 'Robin Haase'], [2014, 'Igor Sijsling', 'Tomas Berdych'], [2014, 'Thiemo De Bakker', 'Tomas Berdych'], [2014, 'Igor Sijsling', 'Lukas Rosol'], [2014, 'Kei Nishikori', 'Peter Polansky']]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [161]:
# Standardizing formats in matches_list and davis_single_list
# Assuming year and match number are integers, and player names are strings

standardized_matches_list = [
    [int(year), str(winner_name), str(loser_name)]
    for year, winner_name, loser_name in matches_list
]

standardized_davis_single_list = [
    [int(year), str(player1), str(player2)]
    for year, player1, player2 in davis_single_list
]

# Convert to sets of tuples for comparison
standardized_matches_set = set(tuple(item) for item in standardized_matches_list)
standardized_davis_single_set = set(tuple(item) for item in standardized_davis_single_list)

# Find non-matching elements in both sets
non_matching_in_standardized_matches = standardized_matches_set - standardized_davis_single_set
non_matching_in_standardized_davis_single = standardized_davis_single_set - standardized_matches_set

print("Non-matching elements in standardized_matches_list:", non_matching_in_standardized_matches)
print("Non-matching elements in standardized_davis_single_list:", non_matching_in_standardized_davis_single)

# Count the number of non-matching tuples in standardized_matches_list
num_unmatched_in_standardized_matches = len(non_matching_in_standardized_matches)

# Count the number of non-matching tuples in standardized_davis_single_list
num_unmatched_in_standardized_davis_single = len(non_matching_in_standardized_davis_single)

print("Number of unmatched elements in standardized_matches_list:", num_unmatched_in_standardized_matches)
print("Number of unmatched elements in standardized_davis_single_list:", num_unmatched_in_standardized_davis_single)


Non-matching elements in standardized_matches_list: set()
Non-matching elements in standardized_davis_single_list: {(2016, 'Ivo Karlovic', 'Juan Martin del Potro'), (2017, 'Albert Ramos', 'Nenad Zimonjic'), (2016, 'Juan Martin del Potro', 'Marin Cilic'), (2018, 'Marc-Andrea Huesler', 'Roman Khassanov'), (2015, 'Jan Lennard Struff', 'Nicolas Mahut'), (2022, 'Felix Auger-Aliassime', 'Oscar Otte'), (2017, 'Albert Ramos', 'Novak Djokovic'), (2019, 'Felix Auger-Aliassime', 'Roberto Bautista Agut'), (2022, 'Denis Shapovalov', 'Jan Lennard Struff'), (2015, 'Gilles Simon', 'Jan Lennard Struff'), (2018, 'Albert Ramos', 'Cameron Norrie'), (2021, 'Daniil Medvedev', 'Jan Lennard Struff'), (2021, 'Cameron Norrie', 'Jan Lennard Struff'), (2019, 'Daniel Evans', 'Jan Lennard Struff'), (2018, 'Jan Lennard Struff', 'Nick Kyrgios'), (2016, 'Andy Murray', 'Juan Martin del Potro'), (2017, 'Pierre-Hugues Herbert', 'Yasutaka Uchiyama'), (2018, 'Albert Ramos', 'Richard Gasquet'), (2018, 'Albert Ramos', 'Liam 

#### names' inconsistency
2015 Jan-Lennard Struff (davis) / Jan Lennard Struff (atp)  
2016 Juan Martin Del Potro (davis) / Juan Martin del Potro (atp)
2017 Albert Ramos-Vinolas (davis) / Albert Ramos (atp)
#### num. obs
331 obs davis cup website
310 obs GitHub repo
21 more obs in davis data

1) check which matches are missing in GitHub repo
2) work on doubles matches

In [162]:
# Assuming standardized_davis_single_list and standardized_matches_list are already created in your environment

# Filter for occurrences of Juan Martin del Potro in standardized_davis_single_list with year == 2016
del_potro_davis_single_2016 = [
    match for match in standardized_davis_single_list 
    if 'Juan Martin del Potro' in match[1:] and match[0] == 2016
]

# Filter for occurrences of Juan Martin del Potro in standardized_matches_list with year == 2016
del_potro_matches_2016 = [
    match for match in standardized_matches_list 
    if 'Juan Martin del Potro' in match[1:] and match[0] == 2016
]

print("Occurrences of Juan Martin del Potro in Davis Single 2016:", del_potro_davis_single_2016)
print("Occurrences of Juan Martin del Potro in Matches 2016:", del_potro_matches_2016)


Occurrences of Juan Martin del Potro in Davis Single 2016: [[2016, 'Andy Murray', 'Juan Martin del Potro'], [2016, 'Ivo Karlovic', 'Juan Martin del Potro'], [2016, 'Juan Martin del Potro', 'Marin Cilic']]
Occurrences of Juan Martin del Potro in Matches 2016: []


there should be occurences for del Potro even in the second df